# py-EM

### a Python module to interact with SerialEM to enable automated Transmission Electron Microscopy

## Table of contents
1. [Installation instructions](#installation)
<br>
  - How to use this jupyter notebook
  - Install necessary dependencies in Python
  - [install/update emTools](#install1)  
<br/>
2. [Getting started](#howtos)
<br>  
  - [Tutorial 1: Sort a Navigator](#tutorial1)
  - [Tutorial 2: Import pixel coordinates](#tutorial2)
  - [Tutorial 3: Duplicate Navigator entries](#tutorial3)   
<br/>


3. [Setting up Integration into SeralEM's GUI](https://git.embl.de/schorb/pyem/blob/master/doc/serialemtools.md)

4. [Setting up KNIME](https://git.embl.de/schorb/pyem/blob/master/doc/knime.md)



## Installation instructions<a name="installation"></a>

### Prepare your PC for the use of Python and py-EM

- first we will install Python and then use it to fetch the packages needed for emTools to work.
- Install Anaconda from [here](https://www.anaconda.com/download/ "Download Anaconda"). EmTools has been tested to work with both Python 2 and 3. Version 3 is recommended as 2.7 is approaching the end of its support frame. Choose the appropriate OS.
- Start the Anaconda [Navigator](http://docs.anaconda.com/anaconda/user-guide/getting-started/ "Getting started with Anaconda").
- as explained in [section 4](http://docs.anaconda.com/anaconda/user-guide/getting-started/#run-python-in-a-jupyter-notebook "Run Python in a Jupyter Notebook"), install and start a Jupyter notebook.
- Download this [tutorial file](https://git.embl.de/schorb/pyem/raw/master/pyEM.ipynb?inline=false) (use "save link as..." if it is displayed in your browser), open it in your Jupyter session and follow the instructions.

----
### How to use this jupyter notebook

In this document there are text blocks and code blocks that can be edited and executed.
In order to run a code block, click into it to activate and then press `CTRL+ Enter`. There is a Python logo at the top right of the notebook that indicates whether there are still operations running.

### Install necessary dependencies in Python

The following code block uses Anaconda to fetch necessary packages. Start the code by pressing `CTRL+ Enter` inside the code block.

On Windows, if you installed Anaconda for all users, you need to run jupyter through Anaconda Navigator with administrative rights.

In [ ]:
# this will load the package needed for python package management
import conda.cli

# this will install the dependencies
conda.cli.main('conda', 'install',  '-y', 'numpy', 'scipy', 'pandas', 'scikit-image', 'git', 'pip')

## install/update py-EM <a name="install1"></a>

Now we can install py-EM using pip, another package manager for Python. Start by pressing `CTRL+ Enter` inside the code block. 

To update to the current version, simply execute this code block anytime.

In [ ]:
# this will load the pip package needed for python package management
from pip._internal import main as pipmain

# this will install the current version of py-EM
pipmain(['install','git+https://git.embl.de/schorb/pyem'])

----



## Getting started <a name="howtos"></a>



### Tutorial 1: Sort a navigator file<a name="tutorial1"></a>

Have a look at the files in the directory ["`tutorials/Tutorial1-sorting`"](https://git.embl.de/schorb/pyem/tree/master/tutorials/Tutorial1-sorting). If you open the navigator file `nav1.nav` in SerialEM, you will get the following:

#### the Navigator
![The navigator](https://git.embl.de/schorb/pyem/raw/master/doc/images/nav1_1.png)

If you double-click the `Map` item, you will see this:
#### the Map
![The cells](https://git.embl.de/schorb/pyem/raw/master/doc/images/nav1.png)

We have a series of points for each cell. However, they are not listed such that all points lying within one cell are acquired in a row.

Let's use emTools to create a sorted Navigator file.


#### Reading in the Navigator

First, we will import the py-EM module and thus make its functions available for the Python script.
Then we will read in the Navigator file. When you have read and understood what each line of the code below does, press `CTRL + Enter` inside the code block to execute it.



In [ ]:
# import the py-EM module and make its functions available
import pyEM as em

# load the navigator file
navfile = 'tutorials/Tutorial1-sorting/nav1.nav'
navlines = em.loadtext(navfile)

# now we have a list of strings for each row of the file, let's look at the first 10.
print('first 10 lines of the navigator: \n')
print(navlines[0:10])
print('\n----\n')


# let's convert them into a list of dictionaries with one entry for each navigator Item
allitems = em.fullnav(navlines)
# Let look at one item
print('Item number 7 from the navigator file: \n')
print(allitems[6])
print('\n----\n')

# Each entry for this item is associated with a dedicated dictionary key. For example the Stage Coordinates
print('Stage coordinates of item number 4: \n')
print(allitems[3]['StageXYZ'])
print('\n----\n')





#### Now, let's find out how to order a navigator list using the function `ordernav`.




In [ ]:
# create a new list of navigator entries using the ordernav function
newnav = em.ordernav(allitems)

print('Let\'s look at the first points in the list...')
print(newnav[1]['# Item'])
print(newnav[2]['# Item'])
print(newnav[3]['# Item'])
print(newnav[4]['# Item'])
print(newnav[5]['# Item'])
print(newnav[6]['# Item'])
print(newnav[7]['# Item'])

# and now we need to write the ordered navigator file.

# create new file by copying the header of the input file
newnavf = navfile[:-4] + '_sorted.nav'
nnf = open(newnavf,'w')
nnf.write("%s\n" % navlines[0])
nnf.write("%s\n" % navlines[1])

# fill the new file   
for nitem in newnav: 
    out = em.itemtonav(nitem,nitem['# Item'])
    for item in out: nnf.write("%s\n" % item)
            
nnf.close()

# done

----

and this is what we get...

![The sorted navigator](https://git.embl.de/schorb/pyem/raw/master/doc/images/nav1_sorted.png)


----

#### Batch process parameters of navigator items

Let's now also try and see if we can change some of the general parameters of these navigator items, like for example z-height, color or the 'Acquire' flag.

In [ ]:
# create new file by copying the header of the input file
newnavf = navfile[:-4] + '_playground.nav'
nnf = open(newnavf,'w')
nnf.write("%s\n" % navlines[0])
nnf.write("%s\n"% navlines[1])

# create a new list to collect ouput navigator items.
outnav=list()

# add the map. This we don't want to modify
outnav.append(newnav[0])

# Then modify and add the rest...

for item in newnav[1:] :
    item['StageXYZ'][2] = '31.415'
    item['Color'] = ['3']
    item['Acquire'] = ['1']
    outnav.append(item)

# fill the new file   
for nitem in outnav: 
    out = em.itemtonav(nitem,nitem['# Item'])
    for item in out: nnf.write("%s\n" % item)
            
nnf.close()

# done

----

and we get...

![The modified navigator](https://git.embl.de/schorb/pyem/raw/master/doc/images/nav1_playground.png)


yellow points at the new height that are set to be acquired.

----





## Tutorial 2: Import pixel coordinates<a name="tutorial2"></a>

This tutorial will give an idea on how to import pixel cooedinates to a map.

In SerialEM, open a new empty Navigator by choosing `Navigator > Open` in the menu.

Then we import a map, this can be any kind of image, maybe from a light microscope if you're doing CLEM. Let's use the SerialEM logo from [this directory](https://git.embl.de/schorb/pyem/tree/master/tutorials/Tutorial2-coordinates). Therefore, click `Navigator > Import Map` and choose a target registration, this is simply the index of the coordinate frame.

![Import Map](https://git.embl.de/schorb/pyem/raw/master/doc/images/import_map.png)
![Import Map1](https://git.embl.de/schorb/pyem/raw/master/doc/images/import_map2.png)

The pixel coordinates of the holes on the top left are stored in [this file](https://git.embl.de/schorb/pyem/tree/master/tutorials/Tutorial2-coordinates/holes.txt). Namely:

| x   | y  |
| --- |---:|
| 13|13|
|32|13|
|51|13|
|13|32|
|32|32|
|13|51|


We now would like to populate our Navigator with those.

In order to tell emTools which map to process, we mark it for acquisition.

![Acquire Map](https://git.embl.de/schorb/pyem/raw/master/doc/images/acquire.png)

Then save the navigator as `nav.nav`.

Now, let's import those into our navigator.



In [ ]:
# import the py-EM module and make its functions available
import emtools as em

# load the navigator file
navfile = 'tutorials/Tutorial2-coordinates/nav.nav'
navlines = em.loadtext(navfile)

allitems = em.fullnav(navlines)

# find the navigator items with active 'Acquire' flag
# first find items that have an 'Acquire' entry
acq = filter(lambda item:item.get('Acquire'),allitems)
acq = list(filter(lambda item:item['Acquire']==['1'],acq))

# in this case, we have only one item
mapitem = acq[0]

# find the Registration of the map as the points shall go into the same
regis = mapitem['Regis']

# print('The target registration is '+ regis[0])

# import the coordinates
importlines =  em.loadtext('tutorials/Tutorial2-coordinates/holes.txt')

# create a navigator
newitems=list()

#remove the Acquire flag from the map item
mapitem['Acquire'] = '0'

newitems.append(mapitem)


# fill the list with new point items
for idx,line in enumerate(importlines):
    
    #fill the relevant fields of the navigator item to create a point with the given label
    point = em.pointitem('Point '+str(idx+1),regis)    
    
    #add the information on which map to place the point
    point['DrawnID'] = mapitem['MapID']
    
    #split the coordinates by the comma separator
    coords = line.split(',')    
          
    # we need to change the y coordinate because of the convention that SerialEM uses
    ymapsize = int(mapitem['MapWidthHeight'][1])
    
    ynew = ymapsize - int(coords[1])
    
    coords[1] = str(ynew)
    
    #add the coordinates, we need an additional z-value for the CoordsInMap entry
    point['PtsX'] = [coords[0]]
    point['PtsY'] = [coords[1]]
    
    coords.append('0')
    point['CoordsInMap'] = coords
    
    newitems.append(point)

# create new file by copying the header of the input file
newnavf = navfile[:-4] + '_with_points.nav'
nnf = open(newnavf,'w')
nnf.write("%s\n" % navlines[0])
nnf.write("%s\n" % navlines[1])

# fill the new file   
for nitem in newitems:
    out = em.itemtonav(nitem,nitem['# Item'])
    for item in out: nnf.write("%s\n" % item)
            
nnf.close()

# done



and with this we get as a result:
![import coordinates](https://git.embl.de/schorb/pyem/raw/master/doc/images/importcoords.png)


----




## Tutorial 3: Duplicate Navigator entries<a name="tutorial3"></a>

This tutorial will give an idea on how to duplicate navigator entries into a new registration. This is needed when propagating items to the following section.

In SerialEM, set the items you like to duplicate to "Acquire".

![Acquire Map](https://git.embl.de/schorb/serialem_tools/raw/master/images/acquire.png)

Then run the following script.

In [ ]:
# import the py-EM module and make its functions available
import pyEM as em

# load the navigator file
navfile = 'E:\\Data\\Isabella\\4091_181216\\grid-1\\s3_sorted.nav'
navlines = em.loadtext(navfile)

allitems = em.fullnav(navlines)


# we will duplicate all items tagged with 'Acquire'
# this is the default selection when using duplicate_items

dup_items = em.duplicate_items(allitems)

# create new file by copying the header of the input file
newnavf = navfile[:-4] +'_duplicates.nav'
nnf = open(newnavf,'w')
nnf.write("%s\n" % navlines[0])
nnf.write("%s\n" % navlines[1])

# fill the new file   
for nitem in dup_items:
    out = em.itemtonav(nitem,nitem['# Item'])
    for item in out: nnf.write("%s\n" % item)
            
nnf.close()

